In [0]:
!nvidia-smi

Sat Apr 27 04:20:11 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    19W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
from google.colab import drive
drive.mount('/Drive')

Drive already mounted at /Drive; to attempt to forcibly remount, call drive.mount("/Drive", force_remount=True).


In [0]:
from osgeo import gdal
import os
import random
import numpy as np
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.models import load_model

path = ('Drive/My Drive/湖北')
class_num = 11
bath_size = 15
steps_per_epoch = 15
epochs = 100

In [0]:
def TrainGenerator_v2(path,batchsize):
    files = os.listdir(path)
    length = files.__len__()
    while (True):
    
        sample = random.sample(range(length),batchsize)
        
        feature = []
        label = []

        for i in sample:
            dataset = gdal.Open(os.path.join(path,files[i]))
            data = dataset.ReadAsArray()
            dataset = None
            data[0]=data[0]%10
            data = np.delete(data,9,axis=0)
            feature.append(data[1:11,:,:])
            label.append(data[0,:,:])
        feature = np.array(feature)
        label = np.array(label)
        label[label>=11]=0
        feature = np.moveaxis(feature, 1, 3)
        label = to_categorical(label,class_num,dtype='uint8')


        yield feature,label

In [0]:
cd ..

/


In [0]:
myGene = TrainGenerator_v2(path=path,batchsize=bath_size)
model = load_model('Drive/My Drive/res_unet.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [0]:
model.compile(optimizer = Adam(lr = 1e-4), loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])
model.fit_generator(myGene,epochs=epochs,steps_per_epoch=steps_per_epoch)
model.save('Drive/My Drive/res_unet.h5')

Epoch 1/300
15/15 [==============================] - 17s 1s/step - loss: 0.5142 - categorical_accuracy: 0.7974
Epoch 2/300
15/15 [==============================] - 11s 762ms/step - loss: 0.5061 - categorical_accuracy: 0.8003
Epoch 3/300
15/15 [==============================] - 12s 788ms/step - loss: 0.4853 - categorical_accuracy: 0.8108
Epoch 4/300
15/15 [==============================] - 12s 804ms/step - loss: 0.4770 - categorical_accuracy: 0.8133
Epoch 5/300
15/15 [==============================] - 12s 783ms/step - loss: 0.4919 - categorical_accuracy: 0.8081
Epoch 6/300
15/15 [==============================] - 11s 766ms/step - loss: 0.4990 - categorical_accuracy: 0.8041
Epoch 7/300
15/15 [==============================] - 11s 757ms/step - loss: 0.4862 - categorical_accuracy: 0.8107
Epoch 8/300
15/15 [==============================] - 11s 755ms/step - loss: 0.4732 - categorical_accuracy: 0.8148
Epoch 9/300
15/15 [==============================] - 11s 755ms/step - loss: 0.4952 - catego